In [1]:
# List of packages with specified versions
packages = [
    "ale-py",
    "AutoROM==0.4.2",
    "AutoROM.accept-rom-license==0.4.2",
    "certifi==2022.12.7",
    "charset-normalizer==3.1.0",
    "click==8.1.3",
    "cloudpickle==2.2.1",
    "cmake==3.26.3",
    "contourpy==1.0.7",
    "cycler==0.11.0",
    "Farama-Notifications==0.0.4",
    "filelock==3.12.0",
    "fonttools==4.39.3",
    "gym==0.23.1",
    "gym-notices==0.0.8",
    "gymnasium-notices==0.0.1",
    "idna==3.4",
    "imageio-ffmpeg==0.4.8",
    "importlib-metadata==6.5.0",
    "importlib-resources==5.12.0",
    "jax-jumpy==1.0.0",
    "Jinja2==3.1.2",
    "kiwisolver==1.4.4",
    "lit==16.0.1",
    "MarkupSafe==2.1.2",
    "matplotlib==3.7.1",
    "mpmath==1.3.0",
    "networkx==3.1",
    "numpy==1.24.2",
    "opencv-python==4.6.0.66",
    "packaging==23.1",
    "pandas==2.0.0",
    "Pillow==9.5.0",
    "pyparsing==3.0.9",
    "python-dateutil==2.8.2",
    "pytz==2023.3",
    "requests==2.28.2",
    "six==1.16.0",
    "stable-baselines3==1.2.0",
    "sympy==1.11.1",
    "torch==2.0.0",
    "torchvision==0.15.1",
    "tqdm==4.65.0",
    "typing_extensions==4.9.0",
    "tzdata==2023.3",
    "urllib3==1.26.15",
    "zipp==3.15.0",
    "moviepy==1.0.3"
]

# Install all packages in one command
!pip install {' '.join(packages)}




  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.7/145.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.2/626.2 kB 16.8 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 9.7 MB/s eta 0:00:00
  

In [2]:
import os
import random
import time

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Imports all our hyperparameters from the other file
# import hyperparams
# from hyperparams import Hyperparameters as params

# stable_baselines3 have wrappers that simplifies
# the preprocessing a lot, read more about them here:
# https://stable-baselines3.readthedocs.io/en/master/common/atari_wrappers.html
from stable_baselines3.common.atari_wrappers import (
    ClipRewardEnv,
    EpisodicLifeEnv,
    FireResetEnv,
    MaxAndSkipEnv,
    NoopResetEnv,
)
from stable_baselines3.common.buffers import ReplayBuffer

/opt/conda/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
2024-05-09 16:32:17.545568: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 16:32:17.545716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 16:32:17.694889: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
class params:
    env_id = "BreakoutNoFrameskip-v4" 
    exp_name = "DQN_Breakout"
    seed = 1
    torch_deterministic = True 
    capture_video = True
    save_model = True
    
    total_timesteps = 10_000_000
    learning_rate = 1e-4
    buffer_size = 1_000_000
    gamma = 0.99
    tau = 1
    target_network_frequency = 1000
    batch_size = 32
    start_e = 1
    end_e = 0.01
    exploration_fraction = 0.1
    learning_starts = 80_000
    train_frequency = 4

In [4]:
# Creates our gym environment and with all our wrappers.
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        env = gym.make(env_id)
#         env = gym.make(env_id, render_mode="rgb_array")
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            if idx == 0:
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env = EpisodicLifeEnv(env)
        if "FIRE" in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = ClipRewardEnv(env)
        env = gym.wrappers.ResizeObservation(env, (84, 84))
        env = gym.wrappers.GrayScaleObservation(env)
        env = gym.wrappers.FrameStack(env, 4)
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return thunk


class QNetwork(nn.Module):
    def __init__(self, env):
        super().__init__()
        # TODO: Deinfe your network (agent)
        # Look at Section 4.1 in the paper for help: https://arxiv.org/pdf/1312.5602v1.pdf

        self.network = nn.Sequential(
            nn.Conv2d(4, 16, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(16, 32, 4, stride=2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(9*9*32, 512), # (((84-8)/4+1)-4)/2+1 = 9
            nn.ReLU(),
            nn.Linear(512, env.single_action_space.n)
        )

    def forward(self, x):
        return self.network(x / 255.0)


def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [6]:
params.env_id

'BreakoutNoFrameskip-v4'

In [7]:
run_name = f"{params.env_id}__{params.exp_name}__{params.seed}__{int(time.time())}"

random.seed(params.seed)
np.random.seed(params.seed)
torch.manual_seed(params.seed)
torch.backends.cudnn.deterministic = params.torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# env setup
envs = gym.vector.SyncVectorEnv([make_env(params.env_id, params.seed, 0, params.capture_video, run_name)])
assert isinstance(envs.single_action_space, gym.spaces.Discrete), "only discrete action space is supported"

q_network = QNetwork(envs).to(device)
optimizer = optim.Adam(q_network.parameters(), lr=params.learning_rate)
target_network = QNetwork(envs).to(device)
target_network.load_state_dict(q_network.state_dict())

# We’ll be using experience replay memory for training our DQN.
# It stores the transitions that the agent observes, allowing us to reuse this data later.
# By sampling from it randomly, the transitions that build up a batch are decorrelated.
# It has been shown that this greatly stabilizes and improves the DQN training procedure.
rb = ReplayBuffer(
    params.buffer_size,
    envs.single_observation_space,
    envs.single_action_space,
    device,
    optimize_memory_usage=False,
    handle_timeout_termination=True,
)

A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]
/opt/conda/lib/python3.10/site-packages/gym/utils/seeding.py:138: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/opt/conda/lib/python3.10/site-packages/gym/utils/seeding.py:175: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:219: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 56.46GB > 31.93GB
  warnings.warn(


In [10]:
obs = envs.reset()


/opt/conda/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:43: DeprecationWarning: WARN: `env.metadata["render.modes"] is marked as deprecated and will be replaced with `env.metadata["render_modes"]` see https://github.com/openai/gym/pull/2654 for more details
  logger.deprecation(
/opt/conda/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/opt/conda/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:421: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(
/opt/conda/lib/python3.10/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high,

In [ ]:
obs = envs.reset()
for global_step in range(params.total_timesteps):
    # Here we get epsilon for our epislon greedy.
    epsilon = linear_schedule(params.start_e, params.end_e, params.exploration_fraction * params.total_timesteps,
                              global_step)

    if random.random() < epsilon:
        # TODO: sample a random action from the environment
        actions = np.array([envs.single_action_space.sample() for _ in range(envs.num_envs)])
    else:
        # TODO: get q_values from the network you defined, what should the network receive as input?
        q_values = q_network(torch.tensor(obs, device=device, dtype=torch.float32))
        actions = torch.argmax(q_values, dim=1).cpu().numpy()

    # Take a step in the environment
    next_obs, rewards, dones, infos = envs.step(actions)

    # Here we print our reward.
    for info in infos:
        if "episode" in info.keys():
            print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
            break

    # Save data to replay buffer
    real_next_obs = next_obs.copy()
    for idx, d in enumerate(dones):
        if d:
            real_next_obs[idx] = infos[idx]["terminal_observation"]

    # Here we store the transitions in D
    rb.add(obs, real_next_obs, actions, rewards, dones, infos)

    obs = next_obs
    # Training
    if global_step > params.learning_starts:
        if global_step % params.train_frequency == 0:
            # Sample random minibatch of transitions from D
            data = rb.sample(params.batch_size)
            # You can get data with:
            # data.observation, data.rewards, data.dones, data.actions

            with torch.no_grad():
                # Now we calculate the y_j for non-terminal phi.
                # TODO: Calculate max Q
                target_max, _ = target_network(
                    torch.tensor(data.next_observations, device=device, dtype=torch.float32)).max(dim=1)

                # TODO: Calculate the td_target (y_j)             (terminal or non-terminal?)
                td_target = data.rewards + params.gamma * target_max * (1 - data.dones)

            # Calculate old_val "q_network().gather(1, data.actions).squeeze()"
            old_val = q_network(torch.tensor(data.observations, device=device, dtype=torch.float32)).gather(1,
                                                                                                            data.actions).squeeze()
            loss = F.mse_loss(old_val, td_target)

            # perform our gradient decent step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # update target network
        if global_step % params.target_network_frequency == 0:
            for target_network_param, q_network_param in zip(target_network.parameters(), q_network.parameters()):
                target_network_param.data.copy_(
                    params.tau * q_network_param.data + (1.0 - params.tau) * target_network_param.data
                )

if params.save_model:
    import os

    model_path = f"./{run_name}/{params.exp_name}_model.pth"

    if not os.path.exists(model_path):
        os.makedirs(os.path.dirname(model_path), exist_ok=True)

    torch.save(q_network.state_dict(), model_path)
    print(f"model saved to {model_path}")

envs.close()

global_step=138, episodic_return=2.0
global_step=375, episodic_return=3.0
global_step=609, episodic_return=3.0
global_step=768, episodic_return=1.0
global_step=881, episodic_return=0.0
global_step=1193, episodic_return=5.0
global_step=1336, episodic_return=1.0
global_step=1445, episodic_return=0.0
global_step=1663, episodic_return=3.0
global_step=1804, episodic_return=1.0
global_step=2099, episodic_return=5.0
global_step=2314, episodic_return=3.0
global_step=2570, episodic_return=3.0
global_step=2681, episodic_return=0.0
global_step=2947, episodic_return=4.0
global_step=3058, episodic_return=0.0
global_step=3229, episodic_return=2.0
global_step=3438, episodic_return=2.0
global_step=3693, episodic_return=4.0
global_step=3948, episodic_return=3.0
global_step=4059, episodic_return=0.0
global_step=4265, episodic_return=2.0
global_step=4380, episodic_return=0.0


/opt/conda/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:43: DeprecationWarning: WARN: `env.metadata["render.modes"] is marked as deprecated and will be replaced with `env.metadata["render_modes"]` see https://github.com/openai/gym/pull/2654 for more details
  logger.deprecation(
/opt/conda/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/opt/conda/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:421: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(
/opt/conda/lib/python3.10/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high,

global_step=4615, episodic_return=3.0
global_step=4803, episodic_return=2.0
global_step=4962, episodic_return=1.0
global_step=5200, episodic_return=3.0
global_step=5313, episodic_return=0.0
global_step=5428, episodic_return=0.0
global_step=5543, episodic_return=0.0
global_step=5656, episodic_return=0.0
global_step=5991, episodic_return=5.0
global_step=6151, episodic_return=1.0
global_step=6294, episodic_return=1.0
global_step=6482, episodic_return=2.0
global_step=6668, episodic_return=2.0
global_step=6876, episodic_return=2.0
global_step=7113, episodic_return=3.0
global_step=7270, episodic_return=1.0
global_step=7458, episodic_return=2.0
global_step=7571, episodic_return=0.0
global_step=7684, episodic_return=0.0
global_step=7893, episodic_return=2.0
global_step=8032, episodic_return=1.0
global_step=8288, episodic_return=3.0
global_step=8399, episodic_return=0.0
global_step=8514, episodic_return=0.0
global_step=8675, episodic_return=1.0
global_step=8790, episodic_return=0.0
global_step=

/opt/conda/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:43: DeprecationWarning: WARN: `env.metadata["render.modes"] is marked as deprecated and will be replaced with `env.metadata["render_modes"]` see https://github.com/openai/gym/pull/2654 for more details
  logger.deprecation(
/opt/conda/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/opt/conda/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:421: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(
/opt/conda/lib/python3.10/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high,

global_step=39902, episodic_return=0.0
global_step=40045, episodic_return=1.0
global_step=40188, episodic_return=1.0
global_step=40409, episodic_return=3.0
global_step=40520, episodic_return=0.0
global_step=40881, episodic_return=8.0
global_step=40994, episodic_return=0.0
global_step=41107, episodic_return=0.0
global_step=41268, episodic_return=1.0
global_step=41409, episodic_return=1.0
global_step=41520, episodic_return=0.0
global_step=41635, episodic_return=0.0
global_step=41774, episodic_return=1.0
global_step=41887, episodic_return=0.0
global_step=42094, episodic_return=2.0
global_step=42530, episodic_return=10.0
global_step=42641, episodic_return=0.0
global_step=42899, episodic_return=3.0
global_step=43058, episodic_return=1.0
global_step=43169, episodic_return=0.0
global_step=43330, episodic_return=1.0
global_step=43469, episodic_return=1.0
global_step=43645, episodic_return=2.0
global_step=43904, episodic_return=4.0
global_step=44015, episodic_return=0.0
global_step=44128, episo

...

global_step=84421, episodic_return=3.0
global_step=84534, episodic_return=0.0
global_step=84846, episodic_return=5.0
global_step=85017, episodic_return=2.0
global_step=85349, episodic_return=5.0
global_step=85490, episodic_return=1.0
global_step=85605, episodic_return=0.0
global_step=85841, episodic_return=3.0
global_step=85956, episodic_return=0.0
global_step=86117, episodic_return=1.0
global_step=86337, episodic_return=3.0
global_step=86452, episodic_return=0.0
global_step=86683, episodic_return=4.0
global_step=86842, episodic_return=1.0
global_step=87030, episodic_return=2.0
global_step=87188, episodic_return=1.0
global_step=87328, episodic_return=1.0
global_step=87443, episodic_return=0.0
global_step=87584, episodic_return=1.0
global_step=87699, episodic_return=0.0
global_step=87860, episodic_return=1.0
global_step=88046, episodic_return=2.0
global_step=88157, episodic_return=0.0
global_step=88272, episodic_return=0.0
global_step=88383, episodic_return=0.0
global_step=88498, episod

/opt/conda/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:43: DeprecationWarning: WARN: `env.metadata["render.modes"] is marked as deprecated and will be replaced with `env.metadata["render_modes"]` see https://github.com/openai/gym/pull/2654 for more details
  logger.deprecation(
/opt/conda/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/opt/conda/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:421: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(
/opt/conda/lib/python3.10/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high,

global_step=247242, episodic_return=1.0
global_step=247385, episodic_return=1.0
global_step=247740, episodic_return=5.0
global_step=247926, episodic_return=2.0
global_step=248067, episodic_return=1.0
global_step=248180, episodic_return=0.0
global_step=248291, episodic_return=0.0
global_step=248404, episodic_return=0.0
global_step=248692, episodic_return=4.0
global_step=249032, episodic_return=8.0
global_step=249192, episodic_return=1.0
global_step=249330, episodic_return=1.0
global_step=249443, episodic_return=0.0
global_step=249556, episodic_return=0.0
global_step=249869, episodic_return=5.0
global_step=249980, episodic_return=0.0
global_step=250139, episodic_return=1.0
global_step=250327, episodic_return=2.0
global_step=250438, episodic_return=0.0
global_step=250597, episodic_return=1.0
global_step=250712, episodic_return=0.0
global_step=250855, episodic_return=1.0
global_step=251091, episodic_return=3.0
global_step=251252, episodic_return=1.0
global_step=251363, episodic_return=0.0


/opt/conda/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:43: DeprecationWarning: WARN: `env.metadata["render.modes"] is marked as deprecated and will be replaced with `env.metadata["render_modes"]` see https://github.com/openai/gym/pull/2654 for more details
  logger.deprecation(
/opt/conda/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/opt/conda/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:421: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(
/opt/conda/lib/python3.10/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high,

global_step=413068, episodic_return=9.0
global_step=413183, episodic_return=0.0
global_step=413298, episodic_return=0.0
global_step=413583, episodic_return=4.0
global_step=413774, episodic_return=2.0
global_step=414040, episodic_return=4.0
global_step=414262, episodic_return=3.0
global_step=414450, episodic_return=2.0
global_step=414565, episodic_return=0.0
global_step=414708, episodic_return=1.0
global_step=414851, episodic_return=1.0
global_step=414964, episodic_return=0.0
global_step=415134, episodic_return=2.0
global_step=415323, episodic_return=2.0
global_step=415529, episodic_return=2.0
global_step=415672, episodic_return=1.0
global_step=415860, episodic_return=2.0
global_step=415973, episodic_return=0.0
global_step=416088, episodic_return=0.0
global_step=416203, episodic_return=0.0
global_step=416419, episodic_return=3.0
global_step=416532, episodic_return=0.0
global_step=416673, episodic_return=1.0
global_step=416788, episodic_return=0.0
global_step=416899, episodic_return=0.0
